# Exploration

## Prerequisites

In [ ]:
from IPython.display import Markdown
import numpy
import pandas
from src import PROCESSED_DATA_DIR

An *action* is an entry in a pipeline;
it is executed in a job.
An action's `id` is the entry's key.
Keys are unique within pipelines but are not unique between pipelines;
that is, keys are locally unique but not globally unique.
Consequently, an action's primary key is a composite of `id` and `job_id`.

In [ ]:
actions = (
    pandas.read_feather(PROCESSED_DATA_DIR / "actions.feather")
    .set_index(["id", "job_id"])
    .sort_index()
)
assert actions.index.is_unique

A *job* is an execution of an action.

In [ ]:
jobs = (
    pandas.read_feather(PROCESSED_DATA_DIR / "jobs.feather")
    .set_index("id")
    .sort_index()
)
assert jobs.index.is_unique

In [ ]:
Markdown(
    f"""
There are {len(actions):,} actions and {len(jobs):,} jobs.
"""
)

Why isn't there a one-to-one relationship between actions and jobs?

Some pipelines couldn't be parsed, so there are jobs that aren't associated with actions.
This suggests there should be more jobs than actions.
However, one job corresponds to one or more actions because the `run_all` action has been expanded.
This suggests there should be more actions than jobs.

---

An action's `pseudo_id` indicates whether the action was run explicitly (`pseudo_id == id`) or implicitly (`pseudo_id == "run_all"`).

---

A *workspace* is a collection of jobs.
We consider a workspace to be a proxy for a study.

In [ ]:
actions = actions.join(jobs.workspace_id, on="job_id")

In [ ]:
actions.head()

## Analysis

How many times have actions of each type been run?

In [ ]:
actions.groupby("type").size().sort_values(ascending=False).rename("count").to_frame()

We'd expect an action to be run more than once per workspace.
However, how many times is a normal number of times?
Are some types of action run more than other types of action per workspace?

`id`s are unique within workspaces but are not unique between workspaces,
so we group by `workspace_id` and `id`.
For convenience, we also group by `type`.

In [ ]:
num_runs_per_workspace = (
    actions.groupby(["workspace_id", "id", "type"]).size().rename("count")
)

In [ ]:
num_runs_per_workspace.groupby("type").aggregate([numpy.mean, max, min])

> `id`s are unique within workspaces

They're *mostly* unique within workspaces.
In some cases, an `id` has been recycled;
in these cases,
the the action has switched between types.
We'd need to do more analysis to understand the direction, though.

In [ ]:
is_duplicated = num_runs_per_workspace.reset_index().duplicated(
    ["id", "workspace_id"], keep=False
)
num_runs_per_workspace.reset_index().loc[is_duplicated].set_index(
    ["id", "workspace_id", "type"]
).sort_index()